# setPath

In [1]:
from sys import path
path.append(r"../")
path.append(r"../../")

# Twins-553

## single results

In [3]:
import os
import argparse
import pandas as pd
import numpy as np
import torch

from utils import log, CausalDataset
from module.TwinsCBIV import run as run_TwinsCBIV

os.environ["CUDA_VISIBLE_DEVICES"] = '2'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

def get_args():
    argparser = argparse.ArgumentParser(description=__doc__)
    # About run setting !!!!
    argparser.add_argument('--seed',default=2021,type=int,help='The random seed')
    argparser.add_argument('--mode',default='vx',type=str,help='The choice of v/x/vx/xx')
    argparser.add_argument('--rewrite_log',default=False,type=bool,help='Whether rewrite log file')
    argparser.add_argument('--use_gpu',default=True,type=bool,help='The use of GPU')
    # About data setting ~~~~
    argparser.add_argument('--num',default=3321,type=int,help='The num of train\val\test dataset')
    argparser.add_argument('--num_reps',default=10,type=int,help='The num of train\val\test dataset')
    argparser.add_argument('--ate',default=-0.0252,type=float,help='The ate of constant')
    argparser.add_argument('--sc',default=1,type=float,help='The sc')
    argparser.add_argument('--sh',default=-2,type=float,help='The sh')
    argparser.add_argument('--one',default=1,type=int,help='The dim of Instrumental variables V')
    argparser.add_argument('--VX',default=1,type=int,help='The dim of Instrumental variables V')
    argparser.add_argument('--mV',default=5,type=int,help='The dim of Instrumental variables V')
    argparser.add_argument('--mX',default=5,type=int,help='The dim of Confounding variables X')
    argparser.add_argument('--mU',default=3,type=int,help='The dim of Unobserved confounding variables U')
    argparser.add_argument('--storage_path',default='../../Data/',type=str,help='The dir of data storage')
    # Twins
    argparser.add_argument('--twins_alpha',default=0.0001,type=float,help='')
    argparser.add_argument('--twins_lambda',default=0.0001,type=float,help='')
    argparser.add_argument('--twins_twoStage',default=True,type=bool,help='')
    # About Debug or Show
    argparser.add_argument('--verbose',default=1,type=int,help='The level of verbose')
    argparser.add_argument('--epoch_show',default=5,type=int,help='The epochs of show time')
    args = argparser.parse_args(args=[])
    return args

args = get_args()

if args.use_gpu:
    device = torch.device('cuda' if torch.cuda.is_available() and args.use_gpu else "cpu")
    
# set path
which_benchmark = 'Twins_'+'_'.join(str(item) for item in [args.sc, args.sh, args.one, args.VX])
which_dataset = '_'.join(str(item) for item in [args.mV, args.mX, args.mU])
resultDir = args.storage_path + f'/results/{which_benchmark}_{which_dataset}/'
dataDir = f'{args.storage_path}/data/{which_benchmark}/{which_dataset}/'
os.makedirs(os.path.dirname(resultDir), exist_ok=True)
logfile = f'{resultDir}/log.txt'

if args.rewrite_log:
    f = open(logfile,'w')
    f.close()

results = []
alpha = args.twins_alpha
for exp in range(args.num_reps):
    # load data
    train_df = pd.read_csv(dataDir + f'{exp}/{args.mode}/train.csv')
    val_df = pd.read_csv(dataDir + f'{exp}/{args.mode}/val.csv')
    test_df = pd.read_csv(dataDir + f'{exp}/{args.mode}/test.csv')

    train = CausalDataset(train_df, variables = ['u','x','v','z','p','s','m','t','g','y','f','c'])
    val = CausalDataset(val_df, variables = ['u','x','v','z','p','s','m','t','g','y','f','c'])
    test = CausalDataset(test_df, variables = ['u','x','v','z','p','s','m','t','g','y','f','c'])

    res_list = []
    
    # args.twins_twoStage = False
    # args.twins_alpha = 0
    # mse_val, obj_val, final = run_TwinsCBIV(exp, args, dataDir, resultDir, train, val, test, device)
    # res_list = res_list + [obj_val['ate_train'],obj_val['ate_test']]
    
    # args.twins_twoStage = False
    # args.twins_alpha = alpha
    # mse_val, obj_val, final = run_TwinsCBIV(exp, args, dataDir, resultDir, train, val, test, device)
    # res_list = res_list + [obj_val['ate_train'],obj_val['ate_test']]
    
    # args.twins_twoStage = True
    # args.twins_alpha = 0
    # mse_val, obj_val, final = run_TwinsCBIV(exp, args, dataDir, resultDir, train, val, test, device)
    # res_list = res_list + [obj_val['ate_train'],obj_val['ate_test']]
    
    args.twins_twoStage = False
    args.twins_alpha = alpha
    mse_val, obj_val, final = run_TwinsCBIV(exp, args, dataDir, resultDir, train, val, test, device)
    res_list = res_list + [obj_val['ate_train'],obj_val['ate_test']]
    
    res = np.array(res_list) + 0.0252
    results.append(res)

results.append(np.mean(results,0))
results.append(np.std(results,0))
    
res_df = pd.DataFrame(np.array(results),
                       columns=[ alpha+data_cls for alpha in ['CBIV'] for data_cls in ['_train', '_test']]).round(4)
res_df.to_csv(resultDir + f'CBIV_{args.mode}_result.csv', index=False)
res_df

exp:0; lrate:0.0005; alpha: 0.0001; lambda: 0.0001; iterations: 200; reweight: 0
Defining graph...



/home/wyliu/yes/envs/tf-torch/lib/python3.6/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/home/wyliu/yes/envs/tf-torch/lib/python3.6/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


0	Obj: 0.6934,	F: 0.6928,	Cf: 0.6928,	Imb: 0.04,	Val: 0.6928,	ValImb: 0.039,	ValObj: 0.6934,	ate_train: 0.0000,	ate_test: 0.0000	pehe_train: 0.0000,	pehe_test: 0.0000
20	Obj: 0.6579,	F: 0.6574,	Cf: 0.6570,	Imb: 0.003,	Val: 0.6566,	ValImb: 0.0028,	ValObj: 0.6571,	ate_train: 0.0021,	ate_test: 0.0021	pehe_train: 0.0032,	pehe_test: 0.0032
40	Obj: 0.4490,	F: 0.4474,	Cf: 0.4317,	Imb: 0.0056,	Val: 0.4372,	ValImb: 0.0055,	ValObj: 0.4388,	ate_train: -0.0467,	ate_test: -0.0467	pehe_train: 0.0690,	pehe_test: 0.0695
60	Obj: 0.4421,	F: 0.4406,	Cf: 0.4258,	Imb: 0.0033,	Val: 0.4299,	ValImb: 0.0029,	ValObj: 0.4315,	ate_train: -0.0259,	ate_test: -0.0259	pehe_train: 0.0382,	pehe_test: 0.0385
80	Obj: 0.4479,	F: 0.4464,	Cf: 0.4285,	Imb: 0.0053,	Val: 0.4370,	ValImb: 0.0042,	ValObj: 0.4385,	ate_train: -0.0597,	ate_test: -0.0597	pehe_train: 0.0883,	pehe_test: 0.0889
100	Obj: 0.4411,	F: 0.4395,	Cf: 0.4240,	Imb: 0.0071,	Val: 0.4272,	ValImb: 0.0056,	ValObj: 0.4287,	ate_train: -0.0220,	ate_test: -0.0220	pehe_tra

ValueError: too many values to unpack (expected 3)

## param experiment

In [ ]:
import os
import argparse
import pandas as pd
import numpy as np
import torch

from utils import log, CausalDataset
from module.TwinsCBIV import run as run_TwinsCBIV

os.environ["CUDA_VISIBLE_DEVICES"] = '2'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

def get_args():
    argparser = argparse.ArgumentParser(description=__doc__)
    # About run setting !!!!
    argparser.add_argument('--seed',default=2021,type=int,help='The random seed')
    argparser.add_argument('--mode',default='vx',type=str,help='The choice of v/x/vx/xx')
    argparser.add_argument('--rewrite_log',default=False,type=bool,help='Whether rewrite log file')
    argparser.add_argument('--use_gpu',default=True,type=bool,help='The use of GPU')
    # About data setting ~~~~
    argparser.add_argument('--num',default=3321,type=int,help='The num of train\val\test dataset')
    argparser.add_argument('--num_reps',default=10,type=int,help='The num of train\val\test dataset')
    argparser.add_argument('--ate',default=-0.0252,type=float,help='The ate of constant')
    argparser.add_argument('--sc',default=1,type=float,help='The sc')
    argparser.add_argument('--sh',default=-2,type=float,help='The sh')
    argparser.add_argument('--one',default=1,type=int,help='The dim of Instrumental variables V')
    argparser.add_argument('--VX',default=1,type=int,help='The dim of Instrumental variables V')
    argparser.add_argument('--mV',default=5,type=int,help='The dim of Instrumental variables V')
    argparser.add_argument('--mX',default=5,type=int,help='The dim of Confounding variables X')
    argparser.add_argument('--mU',default=3,type=int,help='The dim of Unobserved confounding variables U')
    argparser.add_argument('--storage_path',default='../Data/',type=str,help='The dir of data storage')
    # Twins
    argparser.add_argument('--twins_alpha',default=0.0001,type=float,help='')
    argparser.add_argument('--twins_lambda',default=0.0001,type=float,help='')
    argparser.add_argument('--twins_twoStage',default=True,type=bool,help='')
    # About Debug or Show
    argparser.add_argument('--verbose',default=1,type=int,help='The level of verbose')
    argparser.add_argument('--epoch_show',default=5,type=int,help='The epochs of show time')
    args = argparser.parse_args(args=[])
    return args

args = get_args()

if args.use_gpu:
    device = torch.device('cuda' if torch.cuda.is_available() and args.use_gpu else "cpu")
    
# set path
which_benchmark = 'Twins_'+'_'.join(str(item) for item in [args.sc, args.sh, args.one, args.VX])
which_dataset = '_'.join(str(item) for item in [args.mV, args.mX, args.mU])
resultDir = args.storage_path + f'/results/{which_benchmark}_{which_dataset}/'
dataDir = f'{args.storage_path}/data/{which_benchmark}/{which_dataset}/'
os.makedirs(os.path.dirname(resultDir), exist_ok=True)
logfile = f'{resultDir}/log.txt'

if args.rewrite_log:
    f = open(logfile,'w')
    f.close()

results = []
alpha_list = [0, 1e-4, 1e-3, 1e-2, 1e-1, 1]
for exp in range(args.num_reps):
    # load data
    train_df = pd.read_csv(dataDir + f'{exp}/{args.mode}/train.csv')
    val_df = pd.read_csv(dataDir + f'{exp}/{args.mode}/val.csv')
    test_df = pd.read_csv(dataDir + f'{exp}/{args.mode}/test.csv')

    train = CausalDataset(train_df, variables = ['u','x','v','z','p','s','m','t','g','y','f','c'])
    val = CausalDataset(val_df, variables = ['u','x','v','z','p','s','m','t','g','y','f','c'])
    test = CausalDataset(test_df, variables = ['u','x','v','z','p','s','m','t','g','y','f','c'])

    res_list = []
    
    for alpha in alpha_list:
        args.twins_twoStage = True
        args.twins_alpha = alpha
        mse_val, obj_val, final = run_TwinsCBIV(exp, args, dataDir, resultDir, train, val, test, device)
        res_list = res_list + [obj_val['ate_train'],obj_val['ate_test']]
    
    res = np.array(res_list) + 0.0252
    results.append(res)

results.append(np.mean(results,0))
results.append(np.std(results,0))
    
res_df = pd.DataFrame(np.array(results),
                       columns=[ str(alpha)+data_cls for alpha in alpha_list for data_cls in ['_train', '_test']]).round(4)
res_df.to_csv(resultDir + f'CBIV_{args.mode}_alpha_result.csv', index=False)
res_df

# Twins-580

## single results

In [ ]:
import os
import argparse
import pandas as pd
import numpy as np
import torch

from utils import log, CausalDataset
from module.TwinsCBIV import run as run_TwinsCBIV

os.environ["CUDA_VISIBLE_DEVICES"] = '2'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

def get_args():
    argparser = argparse.ArgumentParser(description=__doc__)
    # About run setting !!!!
    argparser.add_argument('--seed',default=2021,type=int,help='The random seed')
    argparser.add_argument('--mode',default='vx',type=str,help='The choice of v/x/vx/xx')
    argparser.add_argument('--rewrite_log',default=False,type=bool,help='Whether rewrite log file')
    argparser.add_argument('--use_gpu',default=True,type=bool,help='The use of GPU')
    # About data setting ~~~~
    argparser.add_argument('--num',default=3321,type=int,help='The num of train\val\test dataset')
    argparser.add_argument('--num_reps',default=10,type=int,help='The num of train\val\test dataset')
    argparser.add_argument('--ate',default=-0.0252,type=float,help='The ate of constant')
    argparser.add_argument('--sc',default=1,type=float,help='The sc')
    argparser.add_argument('--sh',default=-2,type=float,help='The sh')
    argparser.add_argument('--one',default=1,type=int,help='The dim of Instrumental variables V')
    argparser.add_argument('--VX',default=1,type=int,help='The dim of Instrumental variables V')
    argparser.add_argument('--mV',default=5,type=int,help='The dim of Instrumental variables V')
    argparser.add_argument('--mX',default=8,type=int,help='The dim of Confounding variables X')
    argparser.add_argument('--mU',default=0,type=int,help='The dim of Unobserved confounding variables U')
    argparser.add_argument('--storage_path',default='../Data/',type=str,help='The dir of data storage')
    # Twins
    argparser.add_argument('--twins_alpha',default=0.001,type=float,help='')
    argparser.add_argument('--twins_lambda',default=0.0001,type=float,help='')
    argparser.add_argument('--twins_twoStage',default=True,type=bool,help='')
    # About Debug or Show
    argparser.add_argument('--verbose',default=1,type=int,help='The level of verbose')
    argparser.add_argument('--epoch_show',default=5,type=int,help='The epochs of show time')
    args = argparser.parse_args(args=[])
    return args

args = get_args()

if args.use_gpu:
    device = torch.device('cuda' if torch.cuda.is_available() and args.use_gpu else "cpu")
    
# set path
which_benchmark = 'Twins_'+'_'.join(str(item) for item in [args.sc, args.sh, args.one, args.VX])
which_dataset = '_'.join(str(item) for item in [args.mV, args.mX, args.mU])
resultDir = args.storage_path + f'/results/{which_benchmark}_{which_dataset}/'
dataDir = f'{args.storage_path}/data/{which_benchmark}/{which_dataset}/'
os.makedirs(os.path.dirname(resultDir), exist_ok=True)
logfile = f'{resultDir}/log.txt'

if args.rewrite_log:
    f = open(logfile,'w')
    f.close()

results = []
alpha = args.twins_alpha
for exp in range(args.num_reps):
    # load data
    train_df = pd.read_csv(dataDir + f'{exp}/{args.mode}/train.csv')
    val_df = pd.read_csv(dataDir + f'{exp}/{args.mode}/val.csv')
    test_df = pd.read_csv(dataDir + f'{exp}/{args.mode}/test.csv')

    train = CausalDataset(train_df, variables = ['u','x','v','z','p','s','m','t','g','y','f','c'])
    val = CausalDataset(val_df, variables = ['u','x','v','z','p','s','m','t','g','y','f','c'])
    test = CausalDataset(test_df, variables = ['u','x','v','z','p','s','m','t','g','y','f','c'])

    res_list = []
    
    args.twins_twoStage = False
    args.twins_alpha = 0
    mse_val, obj_val, final = run_TwinsCBIV(exp, args, dataDir, resultDir, train, val, test, device)
    res_list = res_list + [obj_val['ate_train'],obj_val['ate_test']]
    
    args.twins_twoStage = False
    args.twins_alpha = alpha
    mse_val, obj_val, final = run_TwinsCBIV(exp, args, dataDir, resultDir, train, val, test, device)
    res_list = res_list + [obj_val['ate_train'],obj_val['ate_test']]
    
    args.twins_twoStage = True
    args.twins_alpha = 0
    mse_val, obj_val, final = run_TwinsCBIV(exp, args, dataDir, resultDir, train, val, test, device)
    res_list = res_list + [obj_val['ate_train'],obj_val['ate_test']]
    
    args.twins_twoStage = True
    args.twins_alpha = alpha
    mse_val, obj_val, final = run_TwinsCBIV(exp, args, dataDir, resultDir, train, val, test, device)
    res_list = res_list + [obj_val['ate_train'],obj_val['ate_test']]
    
    res = np.array(res_list) + 0.0252
    results.append(res)

results.append(np.mean(results,0))
results.append(np.std(results,0))
    
res_df = pd.DataFrame(np.array(results),
                       columns=[ alpha+data_cls for alpha in ['Direct', 'CFR', 'TwoStage', 'CBIV'] for data_cls in ['_train', '_test']]).round(4)
res_df.to_csv(resultDir + f'CBIV_{args.mode}_result.csv', index=False)
res_df

## param experiment

In [ ]:
import os
import argparse
import pandas as pd
import numpy as np
import torch

from utils import log, CausalDataset
from module.TwinsCBIV import run as run_TwinsCBIV

os.environ["CUDA_VISIBLE_DEVICES"] = '2'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

def get_args():
    argparser = argparse.ArgumentParser(description=__doc__)
    # About run setting !!!!
    argparser.add_argument('--seed',default=2021,type=int,help='The random seed')
    argparser.add_argument('--mode',default='vx',type=str,help='The choice of v/x/vx/xx')
    argparser.add_argument('--rewrite_log',default=False,type=bool,help='Whether rewrite log file')
    argparser.add_argument('--use_gpu',default=True,type=bool,help='The use of GPU')
    # About data setting ~~~~
    argparser.add_argument('--num',default=3321,type=int,help='The num of train\val\test dataset')
    argparser.add_argument('--num_reps',default=10,type=int,help='The num of train\val\test dataset')
    argparser.add_argument('--ate',default=-0.0252,type=float,help='The ate of constant')
    argparser.add_argument('--sc',default=1,type=float,help='The sc')
    argparser.add_argument('--sh',default=-2,type=float,help='The sh')
    argparser.add_argument('--one',default=1,type=int,help='The dim of Instrumental variables V')
    argparser.add_argument('--VX',default=1,type=int,help='The dim of Instrumental variables V')
    argparser.add_argument('--mV',default=5,type=int,help='The dim of Instrumental variables V')
    argparser.add_argument('--mX',default=8,type=int,help='The dim of Confounding variables X')
    argparser.add_argument('--mU',default=0,type=int,help='The dim of Unobserved confounding variables U')
    argparser.add_argument('--storage_path',default='../Data/',type=str,help='The dir of data storage')
    # Twins
    argparser.add_argument('--twins_alpha',default=0.001,type=float,help='')
    argparser.add_argument('--twins_lambda',default=0.0001,type=float,help='')
    argparser.add_argument('--twins_twoStage',default=True,type=bool,help='')
    # About Debug or Show
    argparser.add_argument('--verbose',default=1,type=int,help='The level of verbose')
    argparser.add_argument('--epoch_show',default=5,type=int,help='The epochs of show time')
    args = argparser.parse_args(args=[])
    return args

args = get_args()

if args.use_gpu:
    device = torch.device('cuda' if torch.cuda.is_available() and args.use_gpu else "cpu")
    
# set path
which_benchmark = 'Twins_'+'_'.join(str(item) for item in [args.sc, args.sh, args.one, args.VX])
which_dataset = '_'.join(str(item) for item in [args.mV, args.mX, args.mU])
resultDir = args.storage_path + f'/results/{which_benchmark}_{which_dataset}/'
dataDir = f'{args.storage_path}/data/{which_benchmark}/{which_dataset}/'
os.makedirs(os.path.dirname(resultDir), exist_ok=True)
logfile = f'{resultDir}/log.txt'

if args.rewrite_log:
    f = open(logfile,'w')
    f.close()

results = []
alpha_list = [0, 1e-4, 1e-3, 1e-2, 1e-1, 1]
for exp in range(args.num_reps):
    # load data
    train_df = pd.read_csv(dataDir + f'{exp}/{args.mode}/train.csv')
    val_df = pd.read_csv(dataDir + f'{exp}/{args.mode}/val.csv')
    test_df = pd.read_csv(dataDir + f'{exp}/{args.mode}/test.csv')

    train = CausalDataset(train_df, variables = ['u','x','v','z','p','s','m','t','g','y','f','c'])
    val = CausalDataset(val_df, variables = ['u','x','v','z','p','s','m','t','g','y','f','c'])
    test = CausalDataset(test_df, variables = ['u','x','v','z','p','s','m','t','g','y','f','c'])

    res_list = []
    
    for alpha in alpha_list:
        args.twins_twoStage = True
        args.twins_alpha = alpha
        mse_val, obj_val, final = run_TwinsCBIV(exp, args, dataDir, resultDir, train, val, test, device)
        res_list = res_list + [obj_val['ate_train'],obj_val['ate_test']]
    
    res = np.array(res_list) + 0.0252
    results.append(res)

results.append(np.mean(results,0))
results.append(np.std(results,0))
    
res_df = pd.DataFrame(np.array(results),
                       columns=[ str(alpha)+data_cls for alpha in alpha_list for data_cls in ['_train', '_test']]).round(4)
res_df.to_csv(resultDir + f'CBIV_{args.mode}_alpha_result.csv', index=False)
res_df